In [1]:
import deap
import cma
from deap import cma

from deap import creator , base , tools, algorithms
import numpy as np
import random

# Introduction a DEAP : 
DEAP est une librairie qui permet de prototyper rapidement des algorithmes évolutionnaires et des algorithmes génétiques, elle contient des modes de perturbations, de sélection et de croisement plus ou moins sophistiqués qui sont mis a disposition sous forme d'outils.
Dans cette partie nous allons essayer d'aller en compliquant les choses tout en présentant les différents outils implémentés.

## Création de types : 
Dans deap nous pouvons créer dynamiquement des types en utilisant le créateur. \
Généralement nous n'avons pas beaucoup de types a utiliser sauf pour la programmation génétique, on utilisera également "base" pour manipuler plusieurs classes de bases proposées par DEAP.

In [4]:
"""
La fonction create permet de créer un type, elle prend en paramètres un nom, une classe de base et des attributs
chaque
"""

"""
Les premiers type que l'on va créer serrons des fonctions de fitness, pour se faire nous utiliserons la classe 
Fitness présente dans base et comme attributs additionnels nous ajouterons l'attribut weight qui permet de : 
    - Préciser les poids des différents attributs pour les départager si besoin est
    - Préciser le sens dans lequel on optimise i.e si on souhaite les maximiser ou les minimiser
"""
creator.create("Fitness_Function", base.Fitness, weights=(-1.0,))
creator.create("Multi_Objectif", base.Fitness, weights=(-1.0, 1.0))
# Pour utiliser un type crée on ecris par exemple creator.Fitness_Function

"""
A présent on va créer des individus, pour ce faire nous utiliserons généralement comme classe de base 
la classe liste que nous enrichirons par un attribut fitness qui aura un des types crées précédements.
Dans la partie sur la programmation génétique on verra d'autres types plus exotiques.
"""
creator.create("Individual", list, fitness=creator.Fitness_Function)
#On aurait pu utiliser la classe ndarray dispo sur numpy comme classe de base
creator.create("Individual2", np.ndarray , fitness=creator.Fitness_Function)


## Création et remplissage de la Toolbox : 
La toolbox est l'élément le plus important d'un algorithme évolutionnaire, elle permet de spécifier les différentes fonctions que l'on va utiliser pour perturber les solutions, pour les faire muter etc...
Son remplissage nécessite d'utiliser la fonction register, et la encore pour la remplire nous pourrons nous appuyer sur les outils mis a disposition par deap.\
La toolbox sert également a préciser la structure de la population et comment l'initialiser.

In [5]:
"""
D'abord on crée la population et on l'ajoute a la ToolBox 
"""
#Taille de l'individu 
IND_SIZE = 10


#Initialisation de la toolbox
toolbox = base.Toolbox()

#On enregistre la fonction qui sert a initialiser les individus 
toolbox.register("initialisation_random", random.random)
#On ajoute les individus en précisant le mode d'initialisation, le type la fonction d'initialisation et leur taille 
toolbox.register("population", tools.initRepeat, creator.Individual ,toolbox.initialisation_random, n=IND_SIZE)

In [6]:
"""
Maintenant je vais présenter chaque outils
"""

"""
1/Fonction d'évaluation : 
"""
#On crée un individu pour tester en utilisant "individual"

individu1 = toolbox.population()

print(individu1)
print(individu1.fitness) #Fitness pas encore calculée
print(individu1.fitness.valid) #Quand la fitness n'est pas calculée on dit qu'elle est invalide

#D'abord comment enregistrer une fonction d'évaluation


def evaluate(individual):
    #Votre fonction d'évaluation qui retourne une ou plusieurs valeurs (autant qu'il n'y en a dans weights)
    return 2,

individu1.fitness.values = evaluate(individu1)
#Aprés évaluation 
print (individu1.fitness.valid) #ça passe a valid parce qu'on a rempli le champ values
print (individu1.fitness.values)

#Maintenant on enregistre dans la toolbox comme ça

[0.3834386498013367, 0.31975223738892367, 0.5183476780951206, 0.5941670492311408, 0.17799524889284413, 0.6705787158924904, 0.7902379702637863, 0.2600674971745178, 0.6977746192522122, 0.24662568387602846]
()
False
True
(2.0,)


In [9]:
"""
2/Opérateur de mutation (perturbation) : 
"""
mutant = toolbox.clone(individu1)

individu2, = tools.mutGaussian(mutant, mu=0.0, sigma=0.2, indpb=0.9)

del mutant.fitness.values

print(individu1)
print(individu2)
print(individu1.fitness.valid)
print(individu2.fitness.valid)

[0.3834386498013367, 0.31975223738892367, 0.5183476780951206, 0.5941670492311408, 0.17799524889284413, 0.6705787158924904, 0.7902379702637863, 0.2600674971745178, 0.6977746192522122, 0.24662568387602846]
[0.3834386498013367, 0.301724953100099, 0.7009568442197325, 0.5472161009536348, 0.013170572726819013, 0.34118504604689104, 1.1774869215681898, 0.44768809477961424, 0.8051772250624825, 0.14819571975196244]
True
False


In [ ]:
"""
3/Opérateur de croisement (intensification) : 
"""
child1, child2 = [toolbox.clone(ind) for ind in (individu1, individu2)]
tools.cxBlend(child1, child2, 0.8)
del child1.fitness.values
del child2.fitness.values
print(child1)
print(child2)

In [ ]:
"""
4/Opérateur de selection (intensification) : 
"""
Lambd = 3
MU = 10
selected = tools.selBest([child1, child2], 1)
print(selected)

Maintenant on assemble le tout dans une seule bulle ce qui nous fait notre premier algorithme évolutionnaire, nous considérons le design suivant : 
- Une stratégie de perturbation qui consiste a inverser un bit.
- Une sélection par tournoi
- Un croisement Binaire qui consiste en l'inversion de deux points.

Le probléme que nous allons résoudre pour voire si ça marche est le problème de maximum binaire, la fonction d'évaluation serra donc naturellement la somme des éléments de la liste et on s'attend a la fin a trouver un vecteur de 1.

In [11]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()


toolbox.register("attr_bool", random.randint, 0, 1)


toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_bool, 100)

toolbox.register("population", tools.initRepeat, list, toolbox.individual)


def evaluation_func(individual):
    return sum(individual),


toolbox.register("evaluate", evaluation_func)


# ----------
# Ajout des opérateurs 
# ----------
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)


# ----------

random.seed(64)
# create an initial population of 300 individuals (where
# each individual is a list of integers)
pop = toolbox.population(n=300)
"""
Supposons qu'on adopte un mécanisme connu consistant a assigner des probabilités : 
    -CXPB : Probabilités de croisements.
    -MUTPB : Probabilités de mutations.
Ne pas oublier que les mutation diversifient la recherche donc souvent il ne faut pas en abuser pour une bonne
convergence
"""
CXPB, MUTPB = 0.5, 0.2

print("Start of evolution")
# Evaluer la population
fitnesses = list(map(toolbox.evaluate, pop))
for ind, fit in zip(pop, fitnesses):
    ind.fitness.values = fit
    

print("  Evaluated %i individuals" % len(pop))
# Met les fitnesses dans une liste
fits = [ind.fitness.values[0] for ind in pop]
gen = 0
# Begin the evolution
while max(fits) < 100 and gen < 10000:
    # A new generation
    gen = gen + 1
    print("---- Iteration : %i ----" % gen)
    
    # On sélectionne des individus (supposons que tous les éléments parents participent a la génération de l'offspring)
    offspring = toolbox.select(pop, len(pop))
    
    # On les clone etant donné que les oppérateurs aggissent in place et on récupére une liste de références
    offspring = list(map(toolbox.clone, offspring))
    
    # on croise les éléments paires avec les éléments impaires suivant une probabilité CXPB
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            toolbox.mate(child1, child2)
            #On déléte les fitness, vu que les éléments ont changés elle doit être recalculée
            del child1.fitness.values
            del child2.fitness.values
            
    
    for mutant in offspring:
        #Mutation avec probabilité MUTPB
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values
    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit
    print("  Evaluated %i individuals" % len(invalid_ind))
    # The population is entirely replaced by the offspring
    pop[:] = offspring
    # Gather all the fitnesses in one list and print the stats
    fits = [ind.fitness.values[0] for ind in pop]
    length = len(pop)
    mean = sum(fits) / length
    sum2 = sum(x * x for x in fits)
    std = abs(sum2 / length - mean ** 2) ** 0.5
    print("  Min %s" % min(fits))
    print("  Max %s" % max(fits))
    print("  Avg %s" % mean)
    print("  Std %s" % std)
print("-- End of (successful) evolution --")
best_ind = tools.selBest(pop, 1)[0]
print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))

/home/mohamed/.local/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


Start of evolution
  Evaluated 300 individuals
---- Iteration : 1 ----
  Evaluated 181 individuals
  Min 44.0
  Max 66.0
  Avg 54.833333333333336
  Std 4.349584909952722
---- Iteration : 2 ----
  Evaluated 191 individuals
  Min 47.0
  Max 68.0
  Avg 58.45666666666666
  Std 3.455641120769904
---- Iteration : 3 ----
  Evaluated 199 individuals
  Min 52.0
  Max 68.0
  Avg 60.95333333333333
  Std 2.9024970092816367
---- Iteration : 4 ----
  Evaluated 167 individuals
  Min 47.0
  Max 71.0
  Avg 62.96
  Std 2.907186497858939
---- Iteration : 5 ----
  Evaluated 175 individuals
  Min 57.0
  Max 73.0
  Avg 64.99
  Std 2.8489588741621903
---- Iteration : 6 ----
  Evaluated 168 individuals
  Min 58.0
  Max 74.0
  Avg 66.93333333333334
  Std 2.8051539866624524
---- Iteration : 7 ----
  Evaluated 187 individuals
  Min 59.0
  Max 76.0
  Avg 68.91666666666667
  Std 2.826609669236565
---- Iteration : 8 ----
  Evaluated 171 individuals
  Min 62.0
  Max 76.0
  Avg 70.88666666666667
  Std 2.4455038108513

In [12]:
"""
Un peu trop long ? 
Il existe un autre moyen de coder ces comportements basiques (tellement basiques que même moi j'ai copié des
partie de ce code du code de mon prof qui l'a copié du code de la librairie) et c'est en utilisant le package
algorithms.
"""
pop = toolbox.population(n=300)
log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.1, ngen=1000)

gen	nevals
0  	300   
1  	179   
2  	156   
3  	168   
4  	150   
5  	178   
6  	178   
7  	165   
8  	170   
9  	180   
10 	140   
11 	185   
12 	168   
13 	158   
14 	173   
15 	158   
16 	164   
17 	159   
18 	176   
19 	166   
20 	171   
21 	177   
22 	162   
23 	175   
24 	172   
25 	176   
26 	156   
27 	176   
28 	171   
29 	175   
30 	192   
31 	159   
32 	171   
33 	164   
34 	147   
35 	173   
36 	179   
37 	164   
38 	159   
39 	166   
40 	162   
41 	178   
42 	164   
43 	162   
44 	168   
45 	171   
46 	163   
47 	155   
48 	171   
49 	171   
50 	168   
51 	148   
52 	161   
53 	171   
54 	188   
55 	162   
56 	178   
57 	159   
58 	170   
59 	140   
60 	157   
61 	150   
62 	154   
63 	159   
64 	176   
65 	146   
66 	176   
67 	167   
68 	178   
69 	162   
70 	168   
71 	165   
72 	177   
73 	155   
74 	165   
75 	172   
76 	156   
77 	185   
78 	190   
79 	175   
80 	178   
81 	171   
82 	169   
83 	174   
84 	163   
85 	164   
86 	177   
87 	180   
88 	153   
89 	145   

747	177   
748	161   
749	172   
750	164   
751	160   
752	162   
753	182   
754	142   
755	166   
756	160   
757	168   
758	179   
759	157   
760	163   
761	158   
762	173   
763	179   
764	177   
765	184   
766	174   
767	158   
768	171   
769	168   
770	159   
771	166   
772	172   
773	167   
774	163   
775	153   
776	143   
777	155   
778	168   
779	161   
780	175   
781	165   
782	153   
783	173   
784	162   
785	149   
786	153   
787	165   
788	147   
789	169   
790	160   
791	169   
792	166   
793	168   
794	174   
795	158   
796	141   
797	170   
798	175   
799	165   
800	160   
801	157   
802	162   
803	177   
804	184   
805	153   
806	180   
807	155   
808	162   
809	152   
810	171   
811	156   
812	172   
813	172   
814	160   
815	181   
816	171   
817	164   
818	140   
819	152   
820	175   
821	179   
822	170   
823	178   
824	175   
825	145   
826	159   
827	154   
828	155   
829	155   
830	170   
831	159   
832	168   
833	172   
834	187   
835	163   
836	148   
837	151   

In [13]:
"""
Un peu triste comme display...
On peut le customiser en demandant a l'algorithme de calculer des statistiques grace au package stats.
"""
pop = toolbox.population(n=300)
#Initialiser l'objet stats en lui précisant une fonction lui permettant de savoir pour chaque individu sur quoi
#il est sensé calculer ces stats.

stats = tools.Statistics(key=lambda ind: ind.fitness.values)

#On ajoute des statistiques que l'on veut avoir 
stats.register("moyenne", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)
#Pour l'executer sur une population on pourrait faire compile
#Mais la le but est de le donner a l'algorithme
log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.3, ngen=100,stats=stats)

gen	nevals	moyenne	std    	min	max
0  	300   	49.58  	4.75993	38 	66 
1  	193   	53.6233	4.10789	40 	66 
2  	189   	56.7367	3.68067	48 	69 
3  	205   	59.67  	3.77197	53 	70 
4  	207   	62.3067	3.63492	49 	72 
5  	186   	64.8567	3.32708	54 	73 
6  	192   	67.11  	3.0262 	58 	75 
7  	185   	69.2567	2.69025	59 	75 
8  	160   	70.7467	2.67379	61 	78 
9  	200   	72.3567	2.81711	60 	80 
10 	177   	73.8   	2.63059	64 	80 
11 	203   	74.84  	2.82862	67 	82 
12 	188   	76.49  	2.91603	67 	85 
13 	196   	77.9467	2.82084	69 	85 
14 	188   	79.21  	2.85527	70 	85 
15 	181   	80.6567	2.62529	72 	86 
16 	199   	81.74  	2.78072	72 	88 
17 	196   	83.0267	2.52968	73 	90 
18 	189   	83.9367	2.8471 	73 	92 
19 	188   	85.0433	2.62198	76 	92 
20 	200   	85.87  	2.91544	76 	93 
21 	215   	87.0367	3.08901	77 	95 
22 	196   	88.4333	2.78308	79 	95 
23 	194   	89.4033	2.90069	78 	96 
24 	194   	90.48  	2.68383	81 	97 
25 	216   	91.4433	2.82137	82 	97 
26 	202   	92.6733	2.73373	84 	99 
27 	186   	93.55  	2

In [14]:
"""
On retrouve également les autres algorithmes évolutionnaires connus.
"""
pop = toolbox.population(n=300)
#Décomenter pour tester si vous voulez tester
l= algorithms.eaMuPlusLambda(pop, toolbox, mu=300, lambda_=100, cxpb=0.5, mutpb=0.3, ngen=200, stats=stats)
#l= algorithms.eaMuCommaLambda(pop, toolbox, mu=300, lambda_=400, cxpb=0.5, mutpb=0.3, ngen=200, stats=stats)

gen	nevals	moyenne	std    	min	max
0  	300   	49.7567	4.76139	38 	61 
1  	84    	53.8467	3.40536	43 	61 
2  	72    	56.8733	2.33608	50 	62 
3  	80    	58.7   	1.78606	53 	63 
4  	79    	60.35  	1.4586 	56 	65 
5  	80    	61.8633	1.74107	57 	69 
6  	86    	63.5167	1.95014	60 	69 
7  	85    	65.5067	2.14863	61 	71 
8  	78    	67.35  	1.83507	63 	73 
9  	85    	68.84  	1.51473	65 	73 
10 	87    	70.1667	1.31614	67 	74 
11 	80    	71.2967	1.15844	69 	75 
12 	79    	72.1767	1.01265	70 	75 
13 	80    	73.0433	1.01067	71 	75 
14 	83    	73.92  	1.03936	72 	77 
15 	81    	74.8233	0.979178	73 	79 
16 	82    	75.7667	1.02252 	74 	79 
17 	86    	76.5233	1.07213 	74 	80 
18 	77    	77.3267	0.959143	75 	80 
19 	73    	78.0167	0.806053	76 	80 
20 	77    	78.7633	0.787817	76 	81 
21 	78    	79.4933	0.830636	78 	82 
22 	76    	80.2067	0.933071	78 	84 
23 	82    	81.0367	1.21188 	79 	85 
24 	80    	82.04  	1.38747 	80 	86 
25 	84    	83.2067	1.31553 	80 	86 
26 	83    	84.1733	0.914671	82 	86 
27 	83  

On peut également manipuler des stratégies, par exemple CMA-ES qu'on a vu précédemment a été développé en utilisant deap et par conséquent on peut utiliser cette stratégie :
- generate permet de générer une population a partir d'un élément.
- update permet d'actualiser a partir d'une population évaluée les paramètres permettant de generer des éléments

In [15]:

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)
toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_bool, 100)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
def evaluation_func(individual):
    return sum(individual),
toolbox.register("evaluate", evaluation_func)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

pop = toolbox.population(n=300)

S = cma.Strategy(centroid = pop[0],sigma = 0.01)
toolbox.register("generate", S.generate, creator.Individual)
toolbox.register("update",S.update)

stats = tools.Statistics(key=lambda ind: ind.fitness.values)
#On ajoute des statistiques que l'on veut avoir 
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("max", np.max)

logbook = tools.Logbook()



for g in range(100):
    pop = toolbox.generate()
    evaluate(pop)
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
    toolbox.update(pop)
    record = stats.compile(pop)
    logbook.record(gen=g, **record)
    logbook.header = "gen", "avg", "max"
    print(logbook.stream)

"""
Quelqu'un peut expliquer pourquoi ça marche pas ??
"""


gen	avg    	max    
0  	49.9771	50.2646
1  	50.1415	50.3175
2  	50.2635	50.444 
3  	50.4078	50.5971
4  	50.5287	50.7539
5  	50.6785	50.8414
6  	50.7825	50.9581
7  	50.8967	51.149 
8  	51.0492	51.2316
9  	51.1497	51.3902
10 	51.2721	51.4318
11 	51.3378	51.4808
12 	51.467 	51.743 
13 	51.6362	51.8018
14 	51.7589	51.988 
15 	51.8523	52.0113
16 	51.9359	52.0623
17 	52.0566	52.301 
18 	52.216 	52.4076
19 	52.2953	52.5735
20 	52.5319	52.7448
21 	52.7036	52.9411
22 	52.8454	53.0552
23 	53.0296	53.3355
24 	53.2224	53.4173
25 	53.3881	53.5825
26 	53.5594	53.9574
27 	53.7765	54.0492
28 	53.9861	54.2046
29 	54.2695	54.6361
30 	54.524 	54.9638
31 	54.8216	55.2844
32 	55.0557	55.4215
33 	55.3726	55.66  
34 	55.5218	56.0317
35 	55.8278	56.2555
36 	56.0528	56.6608
37 	56.3077	56.7537
38 	56.6871	57.3045
39 	57.0618	57.5634
40 	57.4684	57.8504
41 	57.7043	58.4023
42 	58.3086	58.6839
43 	58.4584	59.0393
44 	58.9054	59.4381
45 	59.4054	60.0014
46 	59.985 	60.422 
47 	60.1601	60.6529
48 	60.4072	61.063 


"\nQuelqu'un peut expliquer pourquoi ça marche pas ??\n"

### A Nous de jouer : 
On va essayer cette fois ci de prototyper rapidement un algorithme evolutionnaire de notre choix sur le dataset boston en utilisant les mêmes fonctions d'évaluations que précédément et le même réseau de neurones.
Vous choisissez l'algorithme que vous voulez et on voit qui a le meilleur résultat en 1000 itérations.

In [16]:
#La fonction d'évaluation : 
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston
def evaluation_function(w):
    X, y = load_boston(return_X_y=True)
    Network = SimpleNeuralControllerNumpy(13, 1, n_hidden_layers=0, n_neurons_per_hidden=0, params=None)
    Network.set_parameters(w)
    y_pred = Network.predict(X)
    return mean_squared_error(y,y_pred),

#Je vous remet la classe NeuralNetwork pour rappel (et pour éviter de l'importer)
def sigmoid(x):
    return 1./(1 + np.exp(-x))

def tanh(x):
    return np.tanh(x)

class SimpleNeuralControllerNumpy():
    def __init__(self, n_in, n_out, n_hidden_layers=2, n_neurons_per_hidden=5, params=None):
        self.dim_in = n_in
        self.dim_out = n_out
        # if params is provided, we look for the number of hidden layers and neuron per layer into that parameter (a dicttionary)
        if (not params==None):
            if ("n_hidden_layers" in params.keys()):
                n_hidden_layers=params["n_hidden_layers"]
            if ("n_neurons_per_hidden" in params.keys()):
                n_neurons_per_hidden=params["n_neurons_per_hidden"]
        self.n_per_hidden = n_neurons_per_hidden
        self.n_hidden_layers = n_hidden_layers
        self.weights = None 
        self.n_weights = None
        self.init_random_params()
        self.out = np.zeros(n_out)
        #print("Creating a simple mlp with %d inputs, %d outputs, %d hidden layers and %d neurons per layer"%(n_in, n_out,n_hidden_layers, n_neurons_per_hidden))
    def init_random_params(self):
        if(self.n_hidden_layers > 0):
            self.weights = [np.random.random((self.dim_in,self.n_per_hidden))] # In -> first hidden
            self.bias = [np.random.random(self.n_per_hidden)] # In -> first hidden
            for i in range(self.n_hidden_layers-1): # Hidden -> hidden
                self.weights.append(np.random.random((self.n_per_hidden,self.n_per_hidden)))
                self.bias.append(np.random.random(self.n_per_hidden))
            self.weights.append(np.random.random((self.n_per_hidden,self.dim_out))) # -> last hidden -> out
            self.bias.append(np.random.random(self.dim_out))
        else:
            self.weights = [np.random.random((self.dim_in,self.dim_out))] # Single-layer perceptron
            self.bias = [np.random.random(self.dim_out)]
        self.n_weights = np.sum([np.product(w.shape) for w in self.weights]) + np.sum([np.product(b.shape) for b in self.bias])

    def get_parameters(self):
        """
        Returns all network parameters as a single array
        """
        flat_weights = np.hstack([arr.flatten() for arr in (self.weights+self.bias)])
        return flat_weights

    def set_parameters(self, flat_parameters):
        """
        Set all network parameters from a single array
        """
        i = 0 # index
        to_set = []
        self.weights = list()
        self.bias = list()
        if(self.n_hidden_layers > 0):
            # In -> first hidden
            w0 = np.array(flat_parameters[i:(i+self.dim_in*self.n_per_hidden)])
            self.weights.append(w0.reshape(self.dim_in,self.n_per_hidden))
            i += self.dim_in*self.n_per_hidden
            for l in range(self.n_hidden_layers-1): # Hidden -> hidden
                w = np.array(flat_parameters[i:(i+self.n_per_hidden*self.n_per_hidden)])
                self.weights.append(w.reshape((self.n_per_hidden,self.n_per_hidden)))
                i += self.n_per_hidden*self.n_per_hidden
            # -> last hidden -> out
            wN = np.array(flat_parameters[i:(i+self.n_per_hidden*self.dim_out)])
            self.weights.append(wN.reshape((self.n_per_hidden,self.dim_out)))
            i += self.n_per_hidden*self.dim_out
            # Samefor bias now
            # In -> first hidden
            b0 = np.array(flat_parameters[i:(i+self.n_per_hidden)])
            self.bias.append(b0)
            i += self.n_per_hidden
            for l in range(self.n_hidden_layers-1): # Hidden -> hidden
                b = np.array(flat_parameters[i:(i+self.n_per_hidden)])
                self.bias.append(b)
                i += self.n_per_hidden
            # -> last hidden -> out
            bN = np.array(flat_parameters[i:(i+self.dim_out)])
            self.bias.append(bN)
            i += self.dim_out
        else:
            n_w = self.dim_in*self.dim_out
            w = np.array(flat_parameters[:n_w])
            self.weights = [w.reshape((self.dim_in,self.dim_out))]
            self.bias = [np.array(flat_parameters[n_w:])]
        self.n_weights = np.sum([np.product(w.shape) for w in self.weights]) + np.sum([np.product(b.shape) for b in self.bias])
    
    def predict(self,x):
        if(self.n_hidden_layers > 0):
            #Input
            a = np.matmul(x,self.weights[0]) + self.bias[0]
            #y = sigmoid(a)
            y = a
            # hidden -> hidden
            for i in range(1,self.n_hidden_layers-1):
                a = np.matmul(y, self.weights[i]) + self.bias[i]
                y = sigmoid(a)
            # Out
            a = np.matmul(y, self.weights[-1]) + self.bias[-1]
            #out = tanh(a)
            out = a
            return out
        else: # Simple monolayer perceptron
            #return tanh(np.matmul(x,self.weights[0]) + self.bias[0])
            return np.matmul(x,self.weights[0]) + self.bias[0]
        
#Et voila pour rappel comment on initialise les paramètres aléatoirement
Network = SimpleNeuralControllerNumpy(13, 1, n_hidden_layers=0, n_neurons_per_hidden=0, params=None)
Network.init_random_params()
print(Network.get_parameters())

[0.11285459 0.87885189 0.66305985 0.00881539 0.45816237 0.78714343
 0.73880625 0.84990367 0.77127734 0.54847152 0.24734991 0.29874114
 0.19920357 0.61708693]


In [17]:
creator.create("FitnessMax", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)


toolbox.register("attr_bool", random.random)

toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_bool, len(Network.get_parameters()))

toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaluation_function)
toolbox.register("mate", tools.cxSimulatedBinary,eta=15)
toolbox.register("mutate", tools.mutGaussian,mu = 0, sigma=0.01, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

pop = toolbox.population(n=300)

l= algorithms.eaMuPlusLambda(pop, toolbox, mu=300, lambda_=100, cxpb=0.5, mutpb=0.3, ngen=200, stats=stats)


/home/mohamed/.local/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/home/mohamed/.local/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


gen	nevals	avg   	std   	max   
0  	300   	231841	151177	686318
1  	78    	112398	75237.3	380414
2  	84    	54435.7	39293.6	210257
3  	83    	22947.1	18163.3	79032.5
4  	87    	9713.86	8477.86	40737.6
5  	80    	3341.55	4691.98	19216.5
6  	78    	710.413	1000.54	10435.8
7  	82    	465.477	83.7707	951.333
8  	87    	415.781	62.2939	481.879
9  	79    	369.61 	52.3402	481.879
10 	74    	327.27 	44.8149	389.46 
11 	82    	293.325	25.8201	380.789
12 	68    	279.97 	15.708 	291.39 
13 	83    	267.285	18.9612	290.385
14 	83    	252.922	14.4228	287.904
15 	86    	245.588	8.00522	284.294
16 	76    	241.537	11.7445	247.284
17 	87    	233.198	18.303 	246.623
18 	80    	218.265	21.7767	245.554
19 	81    	201.057	15.5316	240.174
20 	85    	190.892	13.1615	238.587
21 	76    	181.045	16.1829	198.596
22 	76    	167.435	14.9696	195.314
23 	82    	157.492	6.25019	189.742
24 	86    	154.703	1.80844	157.118
25 	75    	153.128	1.87784	156.735
26 	81    	151.578	1.70255	154.857
27 	82    	150.317	1.16961	15

KeyboardInterrupt: 

In [18]:
S = cma.Strategy(centroid = pop[0],sigma = 0.01)
toolbox.register("generate", S.generate, creator.Individual)
toolbox.register("update",S.update)

stats = tools.Statistics(key=lambda ind: ind.fitness.values)
#On ajoute des statistiques que l'on veut avoir 
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)

logbook = tools.Logbook()



for g in range(1000):
    pop = toolbox.generate()
    evaluate(pop)
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
    toolbox.update(pop)
    record = stats.compile(pop)
    logbook.record(gen=g, **record)
    logbook.header = "gen", "avg", "min"
    print(logbook.stream)


gen	avg    	min    
0  	175.298	125.311
1  	148.357	126.107
2  	141.307	127.623
3  	147.188	125.598
4  	144.03 	127.372
5  	139.42 	128.718
6  	136.835	127.9  
7  	143.447	127.237
8  	169.495	129.587
9  	147.643	125.571
10 	142.713	127.124
11 	160.375	127.55 
12 	148.138	126.21 
13 	142.326	130.245
14 	146.49 	124.138
15 	150.939	122.476
16 	147.459	120.431
17 	151.074	130.128
18 	158.77 	124.353
19 	212.976	129.883
20 	165.309	118.327
21 	189.12 	123.056
22 	164.19 	123.405
23 	147.451	121.851
24 	196.272	130.663
25 	180.033	124.869
26 	154.803	123.8  
27 	194.514	134.144
28 	190.867	130.735
29 	176.605	124.385
30 	173.876	123.217
31 	195.473	124.482
32 	211.391	119.597
33 	171.456	117.408
34 	160.691	120.327
35 	146.536	117.23 
36 	145.637	120.611
37 	169.146	122.975
38 	139.367	115.66 
39 	169.743	120.08 
40 	144.857	115.799
41 	139.614	117.706
42 	155.682	117.98 
43 	147.158	117.294
44 	138.659	120.336
45 	125.395	115.495
46 	131.675	117.755
47 	127.222	118.777
48 	131.302	117.706


413	26.7556	25.999 
414	26.7199	26.0556
415	26.4202	26.1137
416	26.5194	25.9459
417	27.0341	26.3083
418	26.8859	26.4084
419	26.5633	26.1721
420	26.9759	26.1417
421	26.3902	26.0298
422	26.3579	25.9803
423	26.4633	25.9728
424	26.2381	25.8871
425	26.1071	25.8479
426	26.2722	25.9119
427	26.1704	25.827 
428	26.0202	25.7949
429	26.1373	25.8192
430	26.0217	25.7631
431	25.9695	25.7682
432	25.9696	25.7483
433	25.9416	25.7998
434	25.9671	25.7944
435	25.988 	25.8015
436	25.9641	25.7949
437	25.9268	25.7153
438	25.8872	25.7572
439	25.8265	25.7514
440	25.8469	25.7631
441	25.8217	25.7727
442	25.8403	25.7137
443	25.8393	25.7083
444	25.7914	25.6626
445	25.748 	25.6876
446	25.7486	25.6608
447	25.7485	25.6515
448	25.7384	25.6293
449	25.7298	25.6004
450	25.6932	25.5949
451	25.6485	25.5973
452	25.6586	25.611 
453	25.652 	25.5949
454	25.6472	25.595 
455	25.6365	25.5983
456	25.6098	25.5613
457	25.5914	25.5349
458	25.5854	25.5313
459	25.5778	25.5317
460	25.6183	25.5309
461	25.5744	25.4796
462	25.5855	25.4874


KeyboardInterrupt: 